In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("MPS is available:", torch.backends.mps.is_available())
print("MPS is built:", torch.backends.mps.is_built())

device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

PyTorch version: 2.9.1
MPS is available: True
MPS is built: True
Using device: mps
